# **Extract and Separate Audio for Singing on Instrumentals**

## **Introduction**
This notebook provides a Python-based solution to:
1. Extract audio from video or audio sources (YouTube, URLs, or local files).
2. Separate vocals from instrumentals using the Spleeter library.
3. Organize processed files for easy identification and download as a ZIP archive.

### **How to Use**
1. Choose to provide either:
   - A YouTube URL or direct audio URL, **OR**
   - Upload a local file (video/audio).
2. Choose separation mode: `2 stems` (default) or `5 stems`.
3. Optionally, select to download the outputs as MP3.
4. Download the processed ZIP file containing the outputs.

---

### **Setup and Requirements plus Explanation of Imports**

1. **os**:
   - Used to create directories, manipulate paths, and check for file existence.

2. **shutil**:
   - Provides higher-level operations like copying files or directories.

3. **zipfile**:
   - Enables creation and extraction of ZIP files for bundling processed outputs.

4. **Pathlib**:
   - Makes path manipulations easier and more readable, especially for cross-platform compatibility.

5. **subprocess**:
   - Runs shell commands (e.g., calling external tools like `yt-dlp` and `ffmpeg`).

6. **google.colab.files**:
   - Allows file upload/download directly in Google Colab.

7. **ipywidgets**:
   - Provides interactive UI elements such as text input, file upload, and checkboxes.

8. **IPython.display**:
   - Enables embedding of widgets and dynamic content in Colab.

---

Run the following cells to set up the environment.

In [ ]:
# Install necessary libraries
# yt-dlp: For downloading audio or video from YouTube
!pip install yt-dlp --quiet

# spleeter: For separating audio into stems (e.g., vocals and instruments)
!pip install spleeter tensorflow --quiet

# ffmpeg-python: For handling audio and video conversions
!pip install ffmpeg-python --quiet

# Import necessary Python modules
import os  # For file and directory management
import shutil  # For copying and moving files
import zipfile  # For creating ZIP archives
from pathlib import Path  # For easy path manipulations
import subprocess  # For running shell commands
from google.colab import files  # For file upload/download functionality in Colab
from ipywidgets import widgets  # For creating interactive widgets (text boxes, buttons, etc.)
from IPython.display import display  # For displaying widgets and messages

---

### **Step 1: Define Helper Functions**
These functions handle downloading, extracting, and organizing files.


In [ ]:
# Helper function: Download and extract audio
def download_and_extract_audio(input_source, output_dir="/content/input_files"):
    os.makedirs(output_dir, exist_ok=True)

    # Check if input source is a YouTube URL
    if "youtube.com" in input_source or "youtu.be" in input_source:
        print("Downloading from YouTube using yt-dlp...")
        subprocess.run([
            "yt-dlp", "--extract-audio", "--audio-format", "wav", "-o",
            f"{output_dir}/%(title)s.%(ext)s", input_source
        ])
    # Check if input source is a local file or direct URL
    else:
        file_name = os.path.basename(input_source)
        dest_path = os.path.join(output_dir, file_name)
        if input_source.startswith("http"):
            print("Downloading from URL...")
            subprocess.run(["wget", "-O", dest_path, input_source])
        else:
            print("Using local file...")
            shutil.copy(input_source, dest_path)

        # Extract audio if the file is a video
        if file_name.endswith(('.mp4', '.mov')):
            audio_path = dest_path.rsplit('.', 1)[0] + '.wav'
            print("Extracting audio from video...")
            subprocess.run([
                "ffmpeg", "-i", dest_path, "-ar", "44100", "-ac", "2", "-b:a", "192k", audio_path
            ])
            return audio_path

    return output_dir

# Helper function: Process audio with Spleeter
def separate_audio(input_dir, stems=2, output_dir="/content/processed_files"):
    os.makedirs(output_dir, exist_ok=True)
    print("Separating audio with Spleeter...")
    for file in os.listdir(input_dir):
        input_path = os.path.join(input_dir, file)
        if input_path.endswith('.wav'):
            print(f"Processing file: {input_path}")
            command = [
                "spleeter", "separate", f'"{input_path}"', "-p", f"spleeter:{stems}stems", "-o", f'"{output_dir}"'
            ]
            print("Spleeter command:", " ".join(command))
            result = subprocess.run(" ".join(command), shell=True, capture_output=True, text=True)
            print("Spleeter Output:", result.stdout)
            print("Spleeter Errors:", result.stderr)
            if result.returncode != 0:
                raise RuntimeError(f"Spleeter failed to process {input_path}. See errors above.")
    return output_dir

# Helper function: Convert WAV to MP3
def convert_to_mp3(source_dir, output_dir="/content/processed_files_mp3"):
    os.makedirs(output_dir, exist_ok=True)
    print("Converting WAV files to MP3...")
    for root, _, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.wav'):
                wav_path = os.path.join(root, file)
                mp3_path = os.path.join(output_dir, os.path.splitext(file)[0] + '.mp3')
                subprocess.run([
                    "ffmpeg", "-i", wav_path, "-ar", "44100", "-b:a", "192k", mp3_path
                ])
    return output_dir

# Helper function: Create a ZIP file
def create_zip(source_dir, zip_name="processed_files.zip"):
    zip_path = os.path.join("/content", zip_name)
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for root, _, files in os.walk(source_dir):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, source_dir)
                zipf.write(file_path, arcname)
    return zip_path

---
### **Step 2: User Input and Execution with GUI**
Provide your input source and choose processing options using the graphical interface.

In [ ]:
# GUI inputs
input_source_widget = widgets.Text(
    description='Input URL or Path:',
    placeholder='Enter YouTube URL, file path, or direct audio URL...'
)
file_upload_widget = widgets.FileUpload(
    accept='video/*,audio/*', multiple=False, description='Upload File'
)
stems_widget = widgets.Checkbox(
    value=False,
    description='Split all instruments and vocals (default is voice and Backingtrack)',
    layout=widgets.Layout(width='600px', margin='0 0 10px 0')  # Adjust width and add margin
)
mp3_checkbox = widgets.Checkbox(
    value=False,
    description='Download as MP3',
    layout=widgets.Layout(margin='0 0 10px 0')  # Add margin for spacing
)
process_button = widgets.Button(description='Process', layout=widgets.Layout(margin='10px 0'))
output_label = widgets.Label(value="", layout=widgets.Layout(margin='10px 0'))

# Toggle visibility between URL and file upload
input_toggle = widgets.ToggleButtons(
    options=['Use URL', 'Upload File'],
    description='Input Method:',
    style={'description_width': 'initial'}
)

# Dynamically show/hide widgets based on selection
def toggle_input_method(change):
    if change['new'] == 'Use URL':
        input_source_widget.layout.display = ''
        file_upload_widget.layout.display = 'none'
    else:
        input_source_widget.layout.display = 'none'
        file_upload_widget.layout.display = ''

input_toggle.observe(toggle_input_method, names='value')

# Set initial visibility
input_source_widget.layout.display = ''
file_upload_widget.layout.display = 'none'

# Display widgets
display(input_toggle, input_source_widget, file_upload_widget, stems_widget, mp3_checkbox, process_button, output_label)

# Process function
def on_process_clicked(b):
    input_source = input_source_widget.value if input_toggle.value == 'Use URL' else None
    stems = 5 if stems_widget.value else 2
    download_as_mp3 = mp3_checkbox.value

    # Handle file upload if provided
    if input_toggle.value == 'Upload File' and file_upload_widget.value:
        file_info = next(iter(file_upload_widget.value.values()))
        input_source = os.path.join("/content/input_files", file_info['name'])
        with open(input_source, 'wb') as f:
            f.write(file_info['content'])

    if not input_source:
        output_label.value = "Error: Please provide a valid input URL or upload a file."
        return

    # Process the file
    try:
        audio_dir = download_and_extract_audio(input_source)
        processed_dir = separate_audio(audio_dir, stems=stems)
        if download_as_mp3:
            processed_dir = convert_to_mp3(processed_dir)
            zip_name = "processed_files_mp3.zip"
        else:
            zip_name = "processed_files.zip"
        zip_file = create_zip(processed_dir, zip_name=zip_name)
        output_label.value = f"Processing complete. Download your files: {zip_file}"
        files.download(zip_file)
    except Exception as e:
        output_label.value = f"Error: {str(e)}"

process_button.on_click(on_process_clicked)

ToggleButtons(description='Input Method:', options=('Use URL', 'Upload File'), style=ToggleButtonsStyle(descri…

Text(value='', description='Input URL or Path:', layout=Layout(display=''), placeholder='Enter YouTube URL, fi…

FileUpload(value={}, accept='video/*,audio/*', description='Upload File', layout=Layout(display='none'))

Checkbox(value=False, description='Split all instruments and vocals (default is voice and Backingtrack)', layo…

Checkbox(value=False, description='Download as MP3', layout=Layout(margin='0 0 10px 0'))

Button(description='Process', layout=Layout(margin='10px 0'), style=ButtonStyle())

Label(value='', layout=Layout(margin='10px 0'))

Separating audio with Spleeter...
Processing file: /content/input_files/Lady Gaga, Bruno Mars - Die With A Smile (Official Music Video).wav
Spleeter command: spleeter separate "/content/input_files/Lady Gaga, Bruno Mars - Die With A Smile (Official Music Video).wav" -p spleeter:2stems -o "/content/processed_files"
Spleeter Output: INFO:spleeter:File /content/processed_files/Lady Gaga, Bruno Mars - Die With A Smile (Official Music Video)/vocals.wav written succesfully
INFO:spleeter:File /content/processed_files/Lady Gaga, Bruno Mars - Die With A Smile (Official Music Video)/accompaniment.wav written succesfully

Spleeter Errors: RuntimeError: module was compiled against NumPy C-API version 0x10 (NumPy 1.23) but the running NumPy has C-API version 0xf. Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem.

Converting WAV files

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>